In [1]:
## 尝试做特征
import os
from os.path import join
from tqdm import tqdm
from collections import defaultdict as dd
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import numpy as np
from tqdm import trange
from sklearn.metrics import classification_report, precision_recall_fscore_support, average_precision_score
import logging

import utils
import settings

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

data_dir = settings.DATA_TRACE_DIR

papers_train = utils.load_json('', "paper_source_trace_train_ans.json")
papers_valid = utils.load_json('', "paper_source_trace_valid_wo_ans.json")
papers_test = utils.load_json('',  "paper_source_trace_test_wo_ans.json")


files = []
data_dir = './'
in_dir = join(data_dir, 'paper-xml')
for f in os.listdir(in_dir):
    if f.endswith('.xml'):
        files.append(f)

import re
from collections import Counter
import collections
import json


def clean_text(text):
    return re.sub(r'[^a-zA-Z]+', ' ', text).lower().strip()

import re
def get_pattern(input_string):
    pattern = r'\n\n(.*?)\n\n'

# 执行正则表达式搜索
    match = re.search(pattern, input_string)

    # 检查是否有匹配，如果有，则输出匹配到的内容
    if match:
        return match.group(1).strip()
    else:
        return ""
    
## 读数据-GPT
## 读数据


## test
id_train = [ x['_id'] for x in papers_train]
id_valid = [ x['_id'] for x in papers_valid]
id_test = [ x['_id'] for x in papers_test]



/Users/maggiewang/anaconda3/envs/xeek2/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2024-06-13 08:51:56,779 loading paper_source_trace_train_ans.json ...
2024-06-13 08:51:56,839 paper_source_trace_train_ans.json loaded
2024-06-13 08:51:56,840 loading paper_source_trace_valid_wo_ans.json ...
2024-06-13 08:51:56,851 paper_source_trace_valid_wo_ans.json loaded
2024-06-13 08:51:56,852 loading paper_source_trace_test_wo_ans.json ...
2024-06-13 08:51:56,865 paper_source_trace_test_wo_ans.json loaded


In [8]:
import pandas as pd
train_feat = pd.read_csv('train_feat.csv')
test_feat = pd.read_csv('test_feat.csv')

## XGB
## 用random forest 二分类
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import catboost as cb

y_col = 'label'
x_col = [x for x in train_feat.columns if x != y_col]
x_col = [x for x in x_col if x != 'b_key' and x != 'paper_key' and x != 'pred' and x != 'positive_ref_ratio' and x != 'journal']
x_col = [x for x in x_col if x != 'positive_count']
x_col = [x for x in x_col if x != 'venue' and x != 'Unnamed: 0']
x_col = [x for x in x_col if x != 'venue_type' and x != 'journal_type' and x != 'context']
x_col = [x for x in x_col if x != 'gemini_res' and x != 'gemini_res_round2' and x!= 'gemini_res_round3']


In [13]:
## load
import sklearn.externals
import joblib

clf_lgb = joblib.load('lgb.pkl')
clf_catboost = joblib.load('catboost.pkl')

pred = clf_lgb.predict_proba(test_feat[x_col])
pred = pred[:,1]
test_feat['pred_lgb'] = pred

pred = clf_catboost.predict_proba(test_feat[x_col])
pred = pred[:,1]
test_feat['pred_cb'] = pred

In [14]:
    test_feat['pred'] = test_feat['pred_lgb'] * 0.4 + test_feat['pred_cb'] * 0.6

    def get_rule_score(row):
        gpt_labels = []
        gpt_shorts = []
        turbo_res = []
        turbo_res_r2 = []
        turbo_res_note = []
        gpt_shorts_v2 = []
        for ii in range(5):
            gpt_shorts.append(row[f'gpt_res_short_{ii}'])
            gpt_labels.append(row[f'gpt_res_label_{ii}'])
            gpt_shorts_v2.append(row[f'gpt_res_short_{ii}_r2'])
        opus_res = row['opus_res']
        for ii in range(10):
            turbo_res.append(row[f'turbo_res_{ii}'])
            turbo_res_r2.append(row[f'turbo_res_r2_{ii}'])
            turbo_res_note.append(row[f'gpt4_res_note_{ii}'])

        gemini_res = row['gemini_res']
        gemini_res_r2 =  row['gemini_res_round2']
        gemini_res_r3 =  row['gemini_res_round3']
        gemini_res_r4 =  row['gemini_res_round4']

        num_inspired_by = row['num_inspired_by']
        num_motivated_by = row['num_motivated_by']
            

        def score2rank(x):
            if x >= 0.9:
                return 3
            elif x >= 0.5:
                return 2
            elif x > 0.4:
                return 1
            else:
                return 0
            
        opus_rank = score2rank(opus_res)
        gemini_rank = score2rank(gemini_res)
        gemini_rank_r2 = score2rank(gemini_res_r2)
        gemini_rank_r3 = score2rank(gemini_res_r3)
        gemini_rank_r4 = score2rank(gemini_res_r4)
        gemini_res_total = [gemini_res, gemini_res_r2, gemini_res_r3, gemini_res_r4]

        turbo_rank_max = np.max([score2rank(x) for x in turbo_res])
        turbo_rank_mean = np.median([score2rank(x) for x in turbo_res])
        turbo_rank_min = np.min([score2rank(x) for x in turbo_res])

        gpt4o_rank_max = np.max([score2rank(x) for x in gpt_shorts])
        gpt4o_rank_mean = np.median([score2rank(x) for x in gpt_shorts])
        gpt4o_rank_min = np.min([score2rank(x) for x in gpt_shorts])

        turbo_rank_max_r2 = np.max([score2rank(x) for x in turbo_res_r2])
        turbo_rank_mean_r2 = np.median([score2rank(x) for x in turbo_res_r2])
        turbo_rank_min_r2 = np.min([score2rank(x) for x in turbo_res_r2])

        turbo_res_note_max = np.max([score2rank(x)  for x in turbo_res_note])
        turbo_res_note_mean = np.median([score2rank(x)  for x in turbo_res_note])
        turbo_res_note_min = np.min([score2rank(x)  for x in turbo_res_note])

        col1 =  [opus_rank] 
        col2 =  [gemini_rank] + [gemini_rank_r3] + [gemini_rank_r2]
        col3 =  [turbo_rank_max] + [gpt4o_rank_max] + [turbo_rank_max_r2] + [turbo_rank_mean_r2] + [turbo_rank_mean] + [gpt4o_rank_mean] 
        col4 = [turbo_res_note_max] + [turbo_res_note_mean] + [gemini_rank_r4]
        
        
        rule_counter1 = Counter(col1)
        rule_counter2 = Counter(col2)
        rule_counter3 = Counter(col3)
        rule_counter4 = Counter(col4)

        num_31, num_21, num_11 = rule_counter1[3] , rule_counter1[2], rule_counter1[1]
        num_32, num_22, num_12 = rule_counter2[3] , rule_counter2[2], rule_counter2[1]
        num_33, num_23, num_13 = rule_counter3[3] , rule_counter3[2], rule_counter3[1]
        num_34, num_24, num_14 = rule_counter4[3] , rule_counter4[2], rule_counter4[1]

        k1 = 2
        k2 = 1
        k3 = 0.5
        k4 = 1

        score1 = num_31 * k1 + num_32 * k2 + num_33 * k3 + num_34 * k4
        score2 = num_21 * k1 + num_22 * k2 + num_23 * k3 + num_24 * k4
        score3 = num_11 * k1 + num_12 * k2 + num_13 * k3 + num_14 * k4
        
        score = score1 * 4 + score2 * 2 + score3 
        
        

        ### ADD Penalty
        total_single_score = gpt_shorts + gpt_labels + gpt_shorts_v2 + turbo_res_note + turbo_res + turbo_res_r2 + gemini_res_total + [opus_res]
        total_single_score = sorted(total_single_score)
        if total_single_score[20] <= 0.2:
             score = score / 4
        
        return score 

    
    test_feat['rule_score'] = test_feat.apply(get_rule_score, axis = 1)


    def rerank(df, rate_rule):
        df['pred_rule'] = df['pred'] + df['rule_score'] * rate_rule
        return df

    test_feat_rule =  test_feat.groupby(['paper_key']).apply(rerank, rate_rule = 0.035).reset_index(drop = True)

/var/folders/56/b87bdh2x1kbcxmgsfhmgh1sh0000gn/T/ipykernel_32330/358292129.py:107: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_feat_rule =  test_feat.groupby(['paper_key']).apply(rerank, rate_rule = 0.035).reset_index(drop = True)


In [15]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# 写入submission
sample_submission = "submission_example_test.json"
sample_submission = json.load(open(sample_submission, "r"))
import copy
sample_submission_submit = copy.deepcopy(sample_submission)

for sub_key in sample_submission.keys():
    my_ans = []
    my_pred_res = test_feat_rule.loc[test_feat_rule['paper_key'] == sub_key]
    num_pred = len(sample_submission[sub_key])
    for i in range(num_pred):
        b_key = 'b' + str(i)
        try:
            #my_ans.append(sigmoid(my_pred_res.loc[my_pred_res['b_key'] == b_key]['pred'].values[0]))
            my_ans.append(sigmoid(my_pred_res.loc[my_pred_res['b_key'] == b_key]['pred_rule'].values[0]))
        except:
            my_ans.append(0)

    sample_submission_submit[sub_key] = my_ans


import json
with open('sample_submission_0607_1_only_inference.json', 'w') as f:
    json.dump(sample_submission_submit, f)

